# Imports

In [2]:
import dbcontroller as dbc
import dataclasses as dc
import datetime
import decimal
import typing

# Config

In [3]:
DB_URL = {
    "sql" :  "sqlite:///example.db",
    "mongo": "mongodb://localhost:27017/test_database"
}

database = dbc.Database(sql=DB_URL["sql"], mongo=DB_URL["mongo"])

# Default Factory

In [4]:
# DateTime Functions
class Date:
    datetime = lambda: datetime.datetime.now()
    date = lambda: datetime.date.today()
    time = lambda: datetime.datetime.now().time()

# Models

In [5]:
@database.model.sql
class Product:
    name: str
    aliases: list[str] | None = None
    stock: int | None = None
    is_available: bool | None = None
    created_on: datetime.datetime = dc.field(default_factory=Date.datetime)
    available_from: datetime.date = dc.field(default_factory=Date.date)
    same_day_shipping_before: datetime.time = dc.field(default_factory=Date.time)
    price: decimal.Decimal | None = None
    notes: list[dbc.Text] = dc.field(default_factory=list)
    is_object: dbc.JSON = dc.field(default_factory=dict)

    async def category(self) -> typing.Optional["Category"]:
        return Category(name="awesome")


@database.model.mongo
class Category:
    name: str

# Manager

In [6]:
manager = database.manage([Product, Category])

# Tests Purposes Only

In [10]:
from sqlalchemy import create_engine

engine = create_engine(DB_URL["sql"])

database.base.metadata.create_all(engine)

# Testing - Mongo

In [11]:
await manager.mongo.Category.create({ "name" : "John Doe" })

Response(data=namespace(_id=ObjectId('6309d6617c375e3049b4581f'), name='John Doe', id='NjMwOWQ2NjE3YzM3NWUzMDQ5YjQ1ODFmOjpjNDUyZWM5MDY5ZTZmMGU4'), error=False, error_message=None, count=0, pages=0)

# Testing - SQL

In [12]:
await manager.sql.Product.create({ "name" : "John Doe" })

Response(data=namespace(_id=1, name='John Doe', stock=None, is_available=None, created_on=None, available_from=None, same_day_shipping_before=None, price=None, is_object=None, category_id=None, id='MTo6YTU1ZTUzMmVhYjAyOGI0Mg=='), error=False, error_message=None, count=0, pages=0)